# Playing Around and Creating my Own VeloGames API
- cleansed data
- merged data for team comparison
- create web scraper for gathering stage data daily from this [website](https://www.letour.fr/en/stage-14)

In [156]:
# Make sure your local files are here
#ls

In [125]:
# Library Imports
import pandas as pd

In [126]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('tdf_stats.csv')
#df

In [147]:
fantasy_teams = pd.read_csv('team_info.csv')
fantasy_teams

,Hot Wheels,Princess Chubb’s Pawfect Cyclists,Pogacrash :o,Strong Calves,Full Throttle All Uphill,Spoke Too Soon,Wheeley Tired
0,Pogačar,Pogačar,Vingegaard,Pogačar,Evenepoel,Pogačar,A Yates
1,Evenepoel,Evenepoel,Almeida,Almeida,Roglič,Almeida,Van Eetvelt
2,Buitrago,Kuss,Kuss,Champoussin,O'Connor,Skjelmose,Mas
3,Alaphilippe,Alaphilippe,Healy,Healy,Skjelmose,L Martinez,C Rodríguez
4,Bauhaus,Philipsen,Milan,Meeus,Philipsen,Merlier,Merlier
5,Wellens,Eenkhoorn,Van Aert,Abrahamsen,Van Der Poel,Nys,Ballerini
6,Buchmann,Märkl,Van Dijke,Politt,Van Aert,Barthe,Nys
7,Laurance,Wellens,Theuns,Ganna,Barré,De Buyst,Mühlberger
8,Lipowitz,Gall,Roglič,Milan,Arensman,Dillier,Pogačar


In [128]:
def transform_rider_data_to_columns(df_single_column, source_column_name='Data'):
    """
    Transforms a single-column DataFrame containing repeating rider data
    (points string, rider name, team name) into a three-column DataFrame.

    Args:
        df_single_column (pd.DataFrame): A DataFrame with a single column
                                         where data is structured as:
                                         Row 0, 3, 6...: Rider Points (e.g., "150 points")
                                         Row 1, 4, 7...: Rider Name
                                         Row 2, 5, 8...: Rider Team
        source_column_name (str): The name of the single column in the input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with 'Points', 'Rider Name', and 'Team' columns.
    """
    # Ensure the input is a DataFrame and has the specified source column
    if not isinstance(df_single_column, pd.DataFrame) or source_column_name not in df_single_column.columns:
        raise ValueError(f"Input must be a pandas DataFrame with a column named '{source_column_name}'.")

    # Get the data from the single column as a list for easier indexing
    data_list = df_single_column[source_column_name].astype(str).tolist()

    extracted_points = []
    rider_names = []
    teams = []

    # Iterate through the data_list with a step of 3
    # This loop correctly picks out each piece of information based on its position
    for i in range(0, len(data_list), 3):
        try:
            # Extract the points string (e.g., "150 points")
            points_str = data_list[i]
            # Extract the rider name
            name = data_list[i+1]
            # Extract the team name
            team = data_list[i+2]

            # Use regex to get just the number from the points string
            # This handles "150 points" -> "150"
            points_numeric_str = pd.Series([points_str]).str.extract(r'(\d+)\s*points').iloc[0, 0]

            extracted_points.append(points_numeric_str)
            rider_names.append(name)
            teams.append(team)

        except IndexError:
            # This handles cases where the data list is not a perfect multiple of 3
            print(f"Warning: Data seems incomplete starting at index {i}. Skipping remaining data.")
            break # Exit the loop if we run out of data prematurely
        except Exception as e:
            print(f"Error processing data at index {i}: {e}. Skipping this block.")
            # Append None or pd.NA for this block if an error occurs during extraction
            extracted_points.append(pd.NA)
            rider_names.append(pd.NA)
            teams.append(pd.NA)


    # Create the new DataFrame from the collected lists
    df_reshaped = pd.DataFrame({
        'Points': extracted_points,
        'Rider Name': rider_names,
        'Team': teams
    })

    # Convert the 'Points' column to integer type.
    # `errors='coerce'` will turn any non-convertible values into NaN.
    # `astype('Int64')` creates a nullable integer column, which can hold NaN.
    df_reshaped['Points'] = pd.to_numeric(df_reshaped['Points'], errors='coerce').astype('Int64')

    return df_reshaped

In [129]:
rider_stats = transform_rider_data_to_columns(df, 'Header')

,Points,Rider Name,Team
0,2289,Tadej Pogačar,UAE Team Emirates - XRG
1,1687,Jonas Vingegaard,Team Visma | Lease a Bike
2,1059,Mathieu Van Der Poel,Alpecin - Deceuninck
3,1000,Oscar Onley,Team Picnic PostNL
4,903,Matteo Jorgenson,Team Visma | Lease a Bike
...,...,...,...
171,2,Matis Louvel,Israel - Premier Tech
172,2,Sébastien Grignard,Lotto
173,2,Eduardo Sepúlveda,Lotto
174,2,Lennert Van Eetvelt,Lotto


In [141]:
# A list of common name prefixes that should be capitalized
name_prefixes = ['van', 'de', 'le', 'la', 'ter', 'der', 'von', 'del']

def correct_name_case(name):
    """
    Capitalizes a full name correctly, including multi-part last names with prefixes.
    e.g., "wout van aert" -> "Wout Van Aert"
    """
    if pd.isna(name):
        return name
    
    parts = str(name).lower().split()
    
    correct_parts = []
    for part in parts:
        if part in name_prefixes:
            correct_parts.append(part.title())
        else:
            correct_parts.append(part.title())
    
    return " ".join(correct_parts)

def extract_last_name(full_name_or_shorthand):
    """
    Extracts the last name from a full name or a shorthand name,
    correctly handling multi-word prefixes like 'Van Der'.
    Assumes the name has already been put into correct title case.
    """
    if pd.isna(full_name_or_shorthand):
        return np.nan
    
    parts = str(full_name_or_shorthand).split()
    
    last_name_parts = []
    i = len(parts) - 1
    while i >= 0:
        word = parts[i]
        last_name_parts.insert(0, word)
        
        # Check if the preceding word is a prefix
        if i > 0 and parts[i-1].lower() not in name_prefixes:
            break
        i -= 1
        
    return " ".join(last_name_parts)

In [142]:
# Create the 'Standardized Name' column by applying the `correct_name_case` function
rider_stats['Standardized Name'] = rider_stats['Rider Name'].apply(correct_name_case)

# Create the 'Last Name' column by applying the `extract_last_name` function
rider_stats['Last Name'] = rider_stats['Standardized Name'].apply(extract_last_name)

print("--- Final rider_stats DataFrame with Corrected Names ---")
print(rider_stats)

--- Final rider_stats DataFrame with Corrected Names ---
     Points            Rider Name                       Team  \
0      2289         Tadej Pogačar    UAE Team Emirates - XRG   
1      1687      Jonas Vingegaard  Team Visma | Lease a Bike   
2      1059  Mathieu Van Der Poel       Alpecin - Deceuninck   
3      1000           Oscar Onley         Team Picnic PostNL   
4       903      Matteo Jorgenson  Team Visma | Lease a Bike   
..      ...                   ...                        ...   
171       2          Matis Louvel      Israel - Premier Tech   
172       2    Sébastien Grignard                      Lotto   
173       2     Eduardo Sepúlveda                      Lotto   
174       2   Lennert Van Eetvelt                      Lotto   
175       2            Steff Cras         Team TotalEnergies   

        Standardized Name     Last Name  
0           Tadej Pogačar       Pogačar  
1        Jonas Vingegaard    Vingegaard  
2    Mathieu Van Der Poel  Van Der Poel  
3     

In [151]:
# --- THE FIX ---
# Use .groupby() to sum the points for any duplicate last names
rider_stats_aggregated = rider_stats.groupby("Last Name")['Points'].sum().reset_index()

# Now, set the index using the aggregated data
points_lookup = rider_stats_aggregated.set_index("Last Name")["Points"]
points_lookup

Last Name
Abrahamsen     304
Ackermann      270
Affini         296
Alaphilippe    313
Albanese        96
              ... 
Woods           92
Wright         125
Wærenskjold    327
Yates          785
Zimmermann      11
Name: Points, Length: 171, dtype: Int64

In [152]:
# A dictionary to store the new DataFrames, where the key is the team name
team_dataframes = {}

# Iterate through each team column in the fantasy_teams DataFrame
for team_name in fantasy_teams.columns:
    # Get the riders' last names for the current team
    riders = fantasy_teams[team_name]

    # Map the last names to the points lookup table.
    # fillna(0) assigns 0 points to any rider not found in the lookup.
    rider_points = riders.map(points_lookup).fillna(0)
    
    # Create the new DataFrame for this specific team
    team_df = pd.DataFrame({
        'Rider': riders,
        'Rider Points': rider_points
    })
    
    # Store the new DataFrame in our dictionary
    team_dataframes[team_name] = team_df

In [154]:
for team in team_dataframes:

    print(f"--- DataFrame for {team} ---")
    print(team_dataframes[team])
    print("\n" + "="*40 + "\n")

--- DataFrame for Hot Wheels ---
         Rider  Rider Points
0      Pogačar          2289
1    Evenepoel           900
2     Buitrago           109
3  Alaphilippe           313
4      Bauhaus           295
5      Wellens           475
6     Buchmann            26
7     Laurance           313
8     Lipowitz           840


--- DataFrame for Princess Chubb’s Pawfect Cyclists ---
         Rider  Rider Points
0      Pogačar          2289
1    Evenepoel           900
2         Kuss           152
3  Alaphilippe           313
4    Philipsen           329
5    Eenkhoorn            83
6        Märkl            57
7      Wellens           475
8         Gall           581


--- DataFrame for Pogacrash :o ---
        Rider  Rider Points
0  Vingegaard          1687
1     Almeida           348
2        Kuss           152
3       Healy           714
4       Milan           865
5    Van Aert           624
6   Van Dijke             8
7      Theuns            24
8      Roglič           570


--- DataFr

In [ ]:
### TODO
-- Graphing points vs stage for riders
-- line plot for showing how the TDF challenge has progressed                        